In [1]:
import keras.backend as K
from Agent2 import Agent2, ReplayBuffer2
from Agent import Agent, ReplayBuffer
from PokerGame import HUPoker
from PokerInterface import PokerInterface, play_against
from TransformVillain import transformStateVillain


2022-01-13 19:35:00.391611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-13 19:35:00.391654: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-01-13 19:35:03.788458: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-13 19:35:03.788487: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-13 19:35:03.788510: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cromi-Lenovo-V15-ADA): /proc/driver/nvidia/version does not exist
2022-01-13 19:35:03.788730: I tensorflow/core/platform/c

In [2]:
lr = 0.001
gamma = 1
n_actions = 12
epsilon = 1
batch_size = 32
input_dims = (102,)
stacksize = 10
fname = 'trainedModels/PokerAI10BB.h5'
layerSizes = (256,128,64, 32, 16)

In [3]:
newAgent = Agent2(lr, gamma, n_actions, 0.2, batch_size, input_dims, stacksize, fname = fname, handEval=True, layerSizes=layerSizes,mem_size=100000)
oldAgent = Agent2(lr, gamma, n_actions, 0, batch_size, input_dims, stacksize, fname = fname, handEval=True, layerSizes=layerSizes,mem_size=0)
newAgent.load_model()
oldAgent.load_model()

In [4]:
stack_hero = stacksize
stack_villain = stacksize
agentHero = newAgent
agentVillain = oldAgent

In [5]:
game = HUPoker(stack_hero, stack_villain, agentHero, agentVillain)

In [ ]:
#normal hand by hand approach in learning
import numpy as np

n_rounds = 1001
score = 0
for i in range(n_rounds):

    done =True

    while done:
        observation, done = game.reset()

    while not done:

        error = False
        action = agentHero.choose_action(observation)
        observation = np.append(observation,[game.pwinHero, game.plooseHero, game.pwinAvg, game.plooseAvg, game.stdWinAvg])

        try: 
            done = game.step(action) #sometimes the chosen actions lead to too many moves on a street (over 20) This happens when players both chose to engage in minraise battles.
                                    #we want to severely punish this behavior.
        except IndexError: #error happens when more than 20 actions are attepted on a single street.
            done = True
            error = True

        observation_ = game.observation #if street changed from 
        observation_ = np.append(observation_,[game.pwinHero, game.plooseHero, game.pwinAvg, game.plooseAvg, game.stdWinAvg])
        if not done:
            reward = 0
        else:
            if error:
                reward = -stacksize
            else:
                reward = 0.5+game.stack_sb - game.starting_hero if game.position == 0 else 1+game.stack_bb - game.starting_hero #to take into account the blinds are posted anyway regardless of action.
        score += reward - 0.5 if game.position == 0 else reward - 1
        avgScore = round(score//(i+0.00001),2)
        agentHero.store_transition(observation, action, reward, observation_, done)
        observation = game.observation
        
        
        loss = agentHero.learn()
        
        
        
    
    print("hands played: ", i+1, "reward round:", reward, "total score:", score, "avg score", avgScore, "epsilon", agentHero.epsilon)
    if i %10 ==0:
        states, actions, rewards, states_, dones = agentHero.memory.sample_buffer(2)
        q_eval = agentHero.q_eval.predict(states)
        pos = states[0:1][0][0]
        street = states[0:1][0][3]
        hand = (states[0:1][0][4],states[0:1][0][10],states[0:1][0][5],states[0:1][0][11])
        pot = states[0:1][0][7]
        board = states[0:1][0][-15:-5]
        print("pos", pos, "street", street)
        print("hand", hand)
        print("pot", pot)
        print("board",board)
        print(q_eval[0:1])


    if i%1000 == 0:
        #agentHero.save_model()





In [6]:
import ipyparallel as ipp
cluster = ipp.Cluster(n=8)
await cluster.start_cluster() # or cluster.start_cluster_sync() without await

Starting 8 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>


<Cluster(cluster_id='1642098916-v8xd', profile='default', controller=<running>, engine_sets=['1642098917'])>

In [7]:
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=8)

100%|█████████████████████████████████████████| 8/8 [00:03<00:00,  2.07engine/s]


In [8]:
from keras import backend as K
import pickle
fname = 'trainedModels/PokerAI10BB.h5'
layerSizes = (256,128,64, 32, 16)

nDec = 10
lr = 0.00001
epsilon = 0# not meaningful for agentLearn, because he is not playing against anyone. Still all parameters must be included in agent.
batch_size = 16
input_dims = (102,)
stacksize = 10
nCores = 8
agentLearn = Agent2(gamma = gamma, n_actions=n_actions,lr = lr, batch_size = batch_size, input_dims=input_dims, stacksize=stacksize, epsilon = 0, fname = fname, layerSizes=layerSizes, mem_size = nDec*nCores)

try:
    agentLearn.load_model()
    print('Model loaded')
except:
    print('no Model loaded')
K.set_value(agentLearn.q_eval.optimizer.learning_rate, lr)
weightsHero = agentLearn.q_eval.get_weights()

agentHero= Agent2(lr = 0.1, gamma=1, n_actions=12,epsilon=0.1, batch_size=16,
        input_dims=(102,), stacksize =stacksize, 
        mem_size=nDec, 
        epsilon_end=0.05,
        fname=fname, handEval=True,layerSizes=layerSizes)
agent_vil= Agent2(lr = 0.1, gamma=1, n_actions=12,epsilon=0, batch_size=16,
    input_dims=(102,), stacksize =stacksize, 
    mem_size=2, 
    epsilon_end=0.05,
    fname=fname,layerSizes=layerSizes)




env = HUPoker(stacksize, stacksize, agentHero, agent_vil)
rc[:]['agentHero']=agentHero
rc[:]['agent_vil']=agent_vil
rc[:]['weightsHero']=weightsHero

#We want a diversity of opponents in the environment to prevent overfitting, and only update the weights of Villains from time to time, to keep the environment stable enough.
#The current weights for villain are saved in the weights file.
try:
    with open("weightsVil10BB.txt", "rb") as weightsFile:   # Unpickling
        newWeights = pickle.load(weightsFile)
    for i in range(len(newWeights)):
        rc[i]['weightsVil']=newWeights[i]

except:
    rc[:]['weightsVil']=weightsHero
    print("exception!")


rc[:]['ReplayBuffer2']= ReplayBuffer2
rc[:]['env']= env
rc[:]['nDec'] = nDec
rc[:]['stacksize']=stacksize


def play(instance):
    import numpy as np

    agentHero.q_eval.set_weights(weightsHero)
    agent_vil.q_eval.set_weights(weightsVil)
    n_rounds = 1
    env.reset()

    for i in range(n_rounds):
        agentHero.memory = ReplayBuffer2(nDec, (107,))

        while agentHero.memory.mem_cntr < nDec:

            done =True

            while done:
                observation, done = env.reset()
                hand = env.holecards

            while not done:

                error = False
                action = agentHero.choose_action(observation)
                observation = np.append(observation,[env.pwinHero, env.plooseHero, env.pwinAvg, env.plooseAvg, env.stdWinAvg])

                try: 
                    done = env.step(action) #sometimes the chosen actions lead to too many moves on a street (over 20) This happens when players both chose to engage in minraise battles.
                                            #we want to severely punish this behavior.
                except IndexError: #error happens when more than 20 actions are attepted on a single street.
                    done = True
                    error = True

                observation_ = env.observation
                observation_ = np.append(observation_,[env.pwinHero, env.plooseHero, env.pwinAvg, env.plooseAvg, env.stdWinAvg])
                if not done:
                    reward = 0
                else:
                    if error:
                        reward = -stacksize
                    else:
                        reward = 0.5+env.stack_sb - env.starting_hero if env.position == 0 else 1+env.stack_bb - env.starting_hero #to take into account the blinds are posted anyway regardless of action.

                agentHero.store_transition(observation, action, reward, observation_, done)
                    #If the agent is playing against himself to learn, we can use villains play to train also. Everything except for villains hand can be derived from the observation. So we only have to save villains cards.

                observation = env.observation


        memoryDict = {'states': agentHero.memory.state_memory[:nDec], 'states_': agentHero.memory.new_state_memory[:nDec], 
                      'rewards':agentHero.memory.reward_memory[:nDec], 'actions': agentHero.memory.action_memory[:nDec], 
                      'terminals': agentHero.memory.terminal_memory[:nDec]}

        agentHero.memory.mem_cntr =0
    return memoryDict

Model loaded


2022-01-13 19:35:28.125265: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ram://f0635e33-2650-494d-bc1c-b77f19daf462/assets
INFO:tensorflow:Assets written to: ram://1c9e41cb-3a3f-43ff-a0cb-4594250b6c30/assets
INFO:tensorflow:Assets written to: ram://f28bfa1e-90ce-49a5-8127-aeedc75a1033/assets
INFO:tensorflow:Assets written to: ram://d9d91878-3a09-481c-b4cf-7749a5596e7f/assets


In [ ]:
import time
import numpy as np
import gc
nCores = 8#how many parallel cores were running.
oldWeightsCount = 1

for j in range (5000):
    print (j)
    memoryList = rc[:].map_sync(play,range(nCores))

    states = []
    states_=[]
    rewards=[]
    actions=[]
    terminals=[]
    
    for i in range(nCores):
        states.append(memoryList[i]['states'])
        states_.append(memoryList[i]['states_'])
        rewards.append(memoryList[i]['rewards'])
        actions.append(memoryList[i]['actions'])
        terminals.append(memoryList[i]['terminals'])
    
    state_memory=np.concatenate(states,axis=0)
    new_state_memory = np.concatenate(states_,axis=0)
    reward_memory = np.concatenate(rewards,axis=0)
    action_memory = np.concatenate(actions,axis=0)
    terminal_memory = np.concatenate(terminals,axis=0)
    
    agentLearn.memory.state_memory = state_memory
    agentLearn.memory.new_state_memory = new_state_memory
    agentLearn.memory.reward_memory = reward_memory
    agentLearn.memory.action_memory = action_memory
    agentLearn.memory.terminal_memory = terminal_memory
    agentLearn.memory.mem_cntr = nCores*nDec
    
    r1 = np.random.randint(0, nCores*nDec)
    r2 = np.random.randint(0, nCores*nDec)
    observation = agentLearn.memory.state_memory[r1]
    print(observation[0:6],agentLearn.q_eval.predict(np.array([observation])))
    observation = agentLearn.memory.state_memory[r2]
    print(observation[0:6],agentLearn.q_eval.predict(np.array([observation])))

 
    agentLearn.learnMass()   
    agentLearn.save_model()
    
    gc.collect()
    weights = agentLearn.q_eval.get_weights()
    rc[:]['weightsHero']=weights #send current weights of heros model to the engines.
    rc[7]['weightsVillain']=weights #one opponent always has the current weights.
    #update villain weights. After some steps, one of the villain networks gets updated.
    frequencyUpdateVillain = 200
    
    if j%frequencyUpdateVillain ==0:
        print("exchanging weights of",oldWeightsCount)
        print (rc[oldWeightsCount]['weightsVil'][0])
        rc[oldWeightsCount]['weightsVil']= weights
        print (rc[oldWeightsCount]['weightsVil'][0])
        if oldWeightsCount == 6:
            oldWeightsCount = 0
        else:
            oldWeightsCount+=1

#save villain weights
weightsVil = rc[:]['weightsVil']
with open("weightsVil10BB.txt", "wb") as weightsFile:   #Pickling
    pickle.dump(weightsVil, weightsFile)



In [13]:
#save villain weights
weightsVil = rc[:]['weightsVil']
with open("weightsVil10BB.txt", "wb") as weightsFile:   #Pickling
    pickle.dump(weightsVil, weightsFile)